# Mini Projet - Extraction des données GeoNames du Burkina Faso

Ce notebook a pour objectif d'extraire et de traiter un sous-ensemble de données à partir du site [GeoNames](http://www.geonames.org/).
Nous allons :
1. Télécharger et extraire les données du Burkina Faso.
2. Garder uniquement les colonnes ID, location_name, lat, long.
3. Effectuer des filtres spécifiques.
4. Sauvegarder les résultats en CSV et Excel.
5. Permettre le téléchargement direct des fichiers.

In [ ]:

# Installation des bibliothèques nécessaires
!pip install pandas openpyxl


In [ ]:

import pandas as pd
import zipfile
import requests
from google.colab import files

# Télécharger le fichier GeoNames BF.zip
url = "https://download.geonames.org/export/dump/BF.zip"
r = requests.get(url)
with open("BF.zip", "wb") as f:
    f.write(r.content)

# Extraire BF.txt
with zipfile.ZipFile("BF.zip", "r") as zip_ref:
    zip_ref.extractall()


In [ ]:

# Charger et filtrer les colonnes nécessaires
df = pd.read_csv("BF.txt", sep="\t", header=None, dtype=str)
df = df[[0, 1, 4, 5]]
df.columns = ['ID', 'location_name', 'lat', 'long']

# Sauvegarder burkina_location.csv
df.to_csv("burkina_location.csv", index=False)

# Aperçu
df.head()


In [ ]:

# Extraction des données contenant "gounghin"
gounghin_df = df[df['location_name'].str.lower().str.contains("gounghin")]
gounghin_df.to_csv("gounghin.csv", index=False)

gounghin_df.head()


In [ ]:

# Filtrage noms commençant par A à P
A_to_P_df = df[df['location_name'].str[0].str.upper().between('A', 'P')]

# Latitude et longitude minimales
lat_min = A_to_P_df['lat'].astype(float).min()
lat_min_location = A_to_P_df.loc[A_to_P_df['lat'].astype(float) == lat_min, 'location_name'].values

lon_min = A_to_P_df['long'].astype(float).min()
lon_min_location = A_to_P_df.loc[A_to_P_df['long'].astype(float) == lon_min, 'location_name'].values

print("Latitude minimale :", lat_min, "Lieu(x) :", lat_min_location)
print("Longitude minimale :", lon_min, "Lieu(x) :", lon_min_location)

A_to_P_df.head()


In [ ]:

# Lieux avec lat >= 11 et lon <= 0.5
coords_df = df[(df['lat'].astype(float) >= 11) & (df['long'].astype(float) <= 0.5)]
coords_df.head()


In [ ]:

# Création du fichier Excel avec 2 feuilles
with pd.ExcelWriter("mini_projet.xlsx") as writer:
    gounghin_df.to_excel(writer, sheet_name="gounghin", index=False)
    A_to_P_df.to_excel(writer, sheet_name="A_to_P", index=False)


In [ ]:

# Téléchargement direct
files.download("burkina_location.csv")
files.download("gounghin.csv")
files.download("mini_projet.xlsx")
